# INTERCAAT Interface Detection

In [3]:
## Load Libraries
import os
import gzip
import pandas as pd
from intercaat.interface import InterfaceAnalyzer
from Bio.PDB import PDBParser
from Bio.SeqUtils import seq1

In [4]:
## Intercaat Interface Annotator

def extract_chain_sequence_and_mask(structure, chain_id, interface_residue_ids):
    sequence = ""
    masked_sequence = ""
    if chain_id not in structure[0]:
        return "", ""
    chain = structure[0][chain_id]
    i = 0
    for residue in chain:
        if residue.id[0] != " ":
            continue
        try:
            aa = seq1(residue.get_resname())
        except Exception:
            aa = "X"
        i += 1
        sequence += aa
        if i in interface_residue_ids:
            masked_sequence += f"[{aa}]"
        else:
            masked_sequence += aa
    return sequence, masked_sequence

In [2]:
def extract_res_nums(interface_residues):
    # Try to extract residue numbers from the interface_residues output
    # Adjust this depending on the actual structure of interface_residues
    res_nums = []
    for res in interface_residues:
        # If res is a tuple like ('H', 101, 'LEU'), use res[1]
        # If res is a dict or object, adjust accordingly
        if isinstance(res, tuple) and len(res) > 1:
            res_nums.append(res[1])
        elif hasattr(res, 'id'):
            res_nums.append(res.id[1])
        else:
            try:
                res_nums.append(int(res))
            except Exception:
                pass
    return res_nums

In [10]:
df = pd.read_csv("../sabdab/sabdab_with_sequences.tsv", sep="\t")
output_rows = []
parser = PDBParser(QUIET=True)

# pdbs_path = "pdbs" ## Full PDB directory path
pdbs_path = "../sabdab/pdbs_test" ## Test PDB directory path

In [11]:
## Loop through each row in the DataFrame and perform Intercaat analysis
for i, row in df.iterrows():
    pdb_id = row["pdb"]
    pdb_file = os.path.join(pdbs_path, f"{pdb_id}.pdb.gz")
    if not os.path.exists(pdb_file):
        # print(f"[SKIP] File not found: {pdb_file}")
        continue
    try:
        with gzip.open(pdb_file, "rt") as handle:
            structure = parser.get_structure(pdb_id, handle)
            print(f"[INFO] Successfully parsed {pdb_id}")
    except Exception as e:
        print(f"[SKIP] Error parsing {pdb_id}: {e}")
        continue

    heavy_chain = row["Hchain"]
    light_chain = row["Lchain"]
    antigen_chains = []
    if pd.notna(row["AntigenChains"]):
        antigen_chains = str(row["AntigenChains"]).split(" | ")
    antigen_chain = antigen_chains[0].strip() if antigen_chains else ""

    heavy_res_ids = []
    light_res_ids = []
    try:
        if heavy_chain and antigen_chain:
            analyzer = InterfaceAnalyzer(pdb_file, heavy_chain, [antigen_chain])
            heavy_interface = analyzer.get_interface_residues()
            heavy_res_ids = extract_res_nums(heavy_interface)
        if light_chain and antigen_chain:
            analyzer = InterfaceAnalyzer(pdb_file, light_chain, [antigen_chain])
            light_interface = analyzer.get_interface_residues()
            light_res_ids = extract_res_nums(light_interface)
    except Exception as e:
        print(f"[WARNING] Intercaat failed for {pdb_id}: {e}")

    heavy_seq, heavy_mask = extract_chain_sequence_and_mask(structure, heavy_chain, heavy_res_ids) if heavy_chain else ("", "")
    light_seq, light_mask = extract_chain_sequence_and_mask(structure, light_chain, light_res_ids) if light_chain else ("", "")

    output_rows.append({
        "pdb": pdb_id,
        "Hchain": heavy_chain,
        "Lchain": light_chain,
        "AntigenChains": " | ".join(antigen_chains),
        "HeavySeq": heavy_seq,
        "LightSeq": light_seq,
        "HeavyInterfaceResNums": heavy_res_ids,
        "LightInterfaceResNums": light_res_ids,
        "HeavyInterfaceMask": heavy_mask,
        "LightInterfaceMask": light_mask
    })

    # Save every 20 entries
    # if (i + 1) % 20 == 0:
    #     pd.DataFrame(output_rows).to_csv("sabdab_with_sequences_and_interfaces.tsv", sep="\t", index=False)
    #     print(f"Saved {i + 1} entries to sabdab_with_sequences_and_interfaces.tsv")

# Save any remaining entries at the end
pd.DataFrame(output_rows).to_csv("sabdab_with_sequences_and_interfaces.tsv", sep="\t", index=False)
print("Saved all entries to sabdab_with_sequences_and_interfaces.tsv")

[INFO] Successfully parsed 12e8
[INFO] Successfully parsed 15c8
[INFO] Successfully parsed 1a3r
[WARNING] Intercaat failed for 1a3r: QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull v Qc Qbb Qz
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 339026880  voronoi  Qcoplanar-keep  Qbbound-last  Qz-infinity-point
  _pre-merge  _zero-centrum  Qinterior-keep  Pgood  _max-width 64
  Error-roundoff 2e-13  _one-merge 1.8e-12  Visible-distance 1.2e-12
  U-max-coplanar 1.2e-12  Width-outside 2.4e-12  _wide-facet 7.3e-12
  _maxoutside 2.4e-12

precision problems (corrected unless 'Q0' or an error)
    117 zero divisors during gaussian elimination

The input to qhull appears to be less than 4 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p117(v5):    97     1    81    83
- p43(v4):    94     1    17   9.8
- p0(v3):    89     1    44    18
- p113(v2

In [12]:
heavy_res_ids

[]